<a href="https://colab.research.google.com/github/wisesprout/dl-face-detection/blob/main/face_detection_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- collecting and annotating images
> - collect image from webcam, annotate(draw a bounding box) around our head using 'labelme' library
> - 'labelme' can be used for bounding box, key point annotation, segmentation, etc.
- applying bounding box augmentation using 'albumentations'
> - apply random cropping / changes to brightness / flips / gamma shifts and rgb shifts
- build a deep object detection model
>VGG16
>-classification model for classifying face -> result [0] or [1]
>-regression model for estimating the coordinates for bounding box -> result [x1, y1, x2, y2]
- evaluating model performance
>- binary cross entropy loss for face  classification
>- localization loss for true value - predicted value of bounding box
- detect faces in real time

# 1. Setup and Get Data

1-1) Install dependencies and setup
- labelme - https://github.com/wkentaro/labelme
- albumentations - https://albumentations.ai/docs/

In [ ]:
!pip install labelme tensorflow tensorflow-gpu opencv-python matplotlib albumentations

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 511.7 MB 6.2 kB/s 
     |████████████████████████████████| 511.7 MB 5.0 kB/s 
     |████████████████████████████████| 497.9 MB 17 kB/s 
     |████████████████████████████████| 631 kB 75.4 MB/s 
  Created wheel for imgaug: filename=imgaug-0.2.6-py3-none-any.whl size=654017 sha256=878e78d4d969fc4b4b3d9cf2bbf693d74651d9663c0481bcfee491c992865559
  Stored in directory: /root/.cache/pip/wheels/89/72/98/3ebfdba1069a9a8eaaa7ae7265cfd67d63ef0197aaee2e5f9c
Successfully built imgaug
  Attempting uninstall: imgaug
    Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9


1-2) Collect images using OpenCV

In [ ]:
import os  # for navigating different file paths
import time  # setting time between capturing images
import uuid  # for creating unique uniform identifier for each image
import cv2

In [ ]:
IMAGES_PATH = os.path.join('data','images')
number_images = 30

In [ ]:
# collect 30 images per run
cap = cv2.VideoCapture(0)
for imgnum in range(number_images):
    print('Collecting image {}'.format(imgnum))
    ret, frame = cap.read()  # ret: return value for whether image capture was successful / frame: the frame itself
    imgname = os.path.join(IMAGES_PATH,f'{str(uuid.uuid1())}.jpg')  # define name of the frame
    cv2.imwrite(imgname, frame)  # write down the image
    cv2.imshow('frame', frame)
    time.sleep(0.5)  # give 0.5 secs between frames

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

error: ignored

1-3) Annotate images with LableMe
- manually label bounding box coordinates for faces in each image, save to 'labels' folder as a json file
- no label for images where face isn't showing

In [ ]:
!labelme  # start labelme

# 2. Review dataset and build image loading function

- view collected data
- split into train/val/test data. model trains from the train data, use validation to inform how we build the neural network(if performance on the train set and validation set differ too much, we need to change the neural network architecture), use test set for the final performance

2-1) Import TF and Deps